## **[ Produced by ]**

[Eunchan Lee of KNU-BrainAI](https://github.com/purang2)


**[ 목차 ]**


---



### Requirements 설치 

In [ ]:
!pip install transformers 
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.1 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 6.6 MB 28.6 MB/s 
     |████████████████████████████████| 596 kB 33.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 117 kB 4.3 MB/s 


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

## Model Loading

Huggingface/transformers 패키지를 통해 토크나이저와 모델을 load 

Model size: 11.5 GB (큰 것 같다 -> 16GB 권장, 고용량의 RAM 필요)
- KoGPT6B-ryan1.5b는 32GB GPU RAM을 최소로 권장 
- KoGPT6B-ryan1.5b-float16는 16GB GPU RAM을 권장


-> it takes about 5 min

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16', # or float32 version: revision=KoGPT6B-ryan1.5b
    bos_token = '[BOS]', eos_token='[EOS]', unk_token = '[UNK]', pad_token='[PAD]', mask_token = '[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
    'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',# or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

_  = model.eval()

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/839 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5G [00:00<?, ?B/s]

### KorQuAD Dataset Load

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.2 MB/s 
     |████████████████████████████████| 140 kB 48.4 MB/s 
     |████████████████████████████████| 1.1 MB 50.3 MB/s 
     |████████████████████████████████| 212 kB 70.7 MB/s 
     |████████████████████████████████| 127 kB 58.6 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 62.0 MB/s 
     |████████████████████████████████| 271 kB 40.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### KLUE-MRC

In [ ]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/mrc/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 17554
    })
    validation: Dataset({
        features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
        num_rows: 5841
    })
})

In [ ]:
klue_mrc = dataset['train']
klue_mrc

Dataset({
    features: ['title', 'context', 'news_category', 'source', 'guid', 'is_impossible', 'question_type', 'question', 'answers'],
    num_rows: 17554
})

In [ ]:
klue_mrc[1]['context']

'부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스사업화 지원사업’ 공모사업에 4개 과제가 선정되어 본격적인 사업 착수에 나선다. 과학기술정보통신부가 주관하는 ‘지역SW서비스사업화 지원사업’은 강소SW기업 및 초기 스타트업의 SW서비스 사업화 지원과 신시장 진출 지원을 통해 기업 경쟁력 강화와 지역경제 활성화를 도모하는 사업이다. 올해부터 2개년으로 진행되며, 국비와 시비, 민자 등 2년간 약 37억원의 예산이 투입된다. 앞서 진흥원은 부산의 미래 먹거리산업인 스마트해양, 지능형기계, 지능정보서비스 분야로 사전 수요조사를 진행했고, 평가를 통해 선정된 5개 과제를 공모사업에 신청했다. 그 결과 부산의 4개 과제가 최종 선정되는 쾌거를 거뒀다. 당 사업은 전국 진흥기관을 대상으로 공모를 시작해, 총 17개 지역에서 42개 과제가 선정되었으며, 4개 과제가 선정된 곳은 부산과 강원지역 뿐이다. 금번 선정된 과제들은 ‘인공지능융합센서와 서보 이송 로봇을 이용한 전단보강재의 자동용접시스템 개발’ 등 총 4개 과제다. 부산시가 지원하고, 부산정보산업진흥원과 지역기업, 대학, 연구소 등이 컨소시엄을 구성하여 기술개발 및 사업화 지원을 추진한다. 2개의 Track으로 구분되는 이번사업은 Track 1(SW중소기업)에서 ㈜에이아이플랫폼, 엔컴(주), Track 2(스타트업)에서는 ㈜토즈, 삼보테크놀로지를 지원한다. ○ ‘Track 1‘의 (주)에이아이플랫폼이 주관기업으로 진행하는 <인공지능 기반 망막 내 아밀로이드 플라크 영상 분석을 통한 치매조기진단 플랫폼 상용화>는 치매 확진의 원인이 되는 중요 단백질(아밀로이드 플라크)을 자체개발 관측장비로 진단한다. 이를 통해 치매를 조기 발견하여, 각종 경제적 비용과 치료 및 예방 등 사회적 문제를 해 결하고 시민들이 쉽게 접근 가능한 실효성 있는 치매관리체계 개발을 목표로 한다. ○ 엔컴(주)이 주관기업으로 참여하는 <AI영상분석 기반 가공철근 생산성 향상 시스템 기술개발 및 사업화>는 산업안전,

In [ ]:

klue_mrc[1]['source']

'acrofan'

In [ ]:
#len(klue_mrc[1]['context'])

klue_mrc[1]['question']





'성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?'

In [ ]:

klue_mrc[1]['answers']


{'answer_start': [18, 21, 1673], 'text': ['원장 이인숙', '이인숙', '이인숙']}

In [ ]:
ex = klue_mrc['title']

len(ex)

17554

In [ ]:
ex = list(set(ex))


len(ex)



12169

### Experimental Setup 


  논문 [Onthe Effect of Pretraining Corpora on In-context Learning by a Large-scale Language Model]의 실험 섹션인 $3.4를 참고한다.

  - Reading Comprehension 태스크인 KorQuAD에서 4 Shot Learning을 적용하였다.




### KorQuAD

In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [ ]:
dataset['validation']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5774
})

In [ ]:
data = dataset['train']

In [ ]:
data[0]

{'answers': {'answer_start': [54], 'text': ['교향곡']},
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'title': '파우스트_서곡'}

In [ ]:
data[50]['answers']['text'][0]

'이북 구간'

In [ ]:
data['context'][52803]   # 52800 ~ 52803

'다음은 미국의 여배우 제시카 채스테인의 작품 목록이다. 줄리아드 스쿨에서 졸업반 학생이었던 그녀는 TV 프로듀서인 존 웰스의 눈에 띄어 계약에 서명했다. 2004년부터 2010년까지 《ER》, 《베로니카 마스》, 《로 앤 오더: 배심원에 의한 재판》등 여러 TV 프로그램에서 게스트 역할로 출연했다. 그녀는 2004년 미셸 윌리엄스와 《벚꽃 동산》, 2006년 알 파치노와 《살로메》와 같은 무대 연극에도 출연했다. 2008년 채스테인은 E. L.닥터로의 단편 소설 Jolene: A Life를 각색한 댄 아일랜드 감독의 드라마 영화 《조렌느》에서 타이틀 롤을 맡으며 영화 데뷔를 하였다. 그녀는 비판적인 평가를 받은 미스테리 스릴러 영화 《스틸》에서 작은 역할을했으며, 그 후 그녀는 《언피니시드》(2010)에서 헬렌 미렌이 분한 레이철 싱어 역할의 더 어린 버전을 연기했다.'

### K=4 Few-shot KorQuAD 

In [ ]:
promptz = '에 대해 답이 뭔지 아니? :\n\n'


#prompt = prompt + f"맥락: {data['context'][52803]} \n\n" 

#prompt 3 
prompt = f"맥락: {data['context'][52803]} \n\n" 
promptz = '에 대해 답이 뭔지 아니? :\n\n'


#Demo with gold label

for i in range(4):
  prompt = prompt + f"질문: {data[52799+i]['question']} {promptz}\n 대답: {data[52799+i]['answers']['text'][0]} [끝]\n\n"



#Demo with random label
'''
labels = [4643, 9999, 12222, 583]
for i in range(4):
  prompt = prompt + f"질문: {data[52800+i]['question']} \n 대답: {data[labels[i]]['answers']['text'][0]} [끝]\n\n"
'''


#No demo (=zero-shot)
'''
prompt= ''
'''


print(prompt)

맥락: 다음은 미국의 여배우 제시카 채스테인의 작품 목록이다. 줄리아드 스쿨에서 졸업반 학생이었던 그녀는 TV 프로듀서인 존 웰스의 눈에 띄어 계약에 서명했다. 2004년부터 2010년까지 《ER》, 《베로니카 마스》, 《로 앤 오더: 배심원에 의한 재판》등 여러 TV 프로그램에서 게스트 역할로 출연했다. 그녀는 2004년 미셸 윌리엄스와 《벚꽃 동산》, 2006년 알 파치노와 《살로메》와 같은 무대 연극에도 출연했다. 2008년 채스테인은 E. L.닥터로의 단편 소설 Jolene: A Life를 각색한 댄 아일랜드 감독의 드라마 영화 《조렌느》에서 타이틀 롤을 맡으며 영화 데뷔를 하였다. 그녀는 비판적인 평가를 받은 미스테리 스릴러 영화 《스틸》에서 작은 역할을했으며, 그 후 그녀는 《언피니시드》(2010)에서 헬렌 미렌이 분한 레이철 싱어 역할의 더 어린 버전을 연기했다. 

질문: 제시카 채스테인의 국적은? 에 대해 답이 뭔지 아니? :


 대답: 미국 [끝]

질문: 제시카 채스테인의 영화 데뷔작은? 에 대해 답이 뭔지 아니? :


 대답: 조렌느 [끝]

질문: 제시카 채스테인은 어느 학교에서 캐스팅 되었는가? 에 대해 답이 뭔지 아니? :


 대답: 줄리아드 스쿨 [끝]

질문: 제시카 채스테인을 처음 발탁한 TV 프로듀서는 누구인가? 에 대해 답이 뭔지 아니? :


 대답: 존 웰스 [끝]




In [ ]:
len(prompt)

688

In [ ]:
eval_data = dataset['validation']

answer_set = [] #answer set 


#prompt 

'''
#1: 아래 지문을 읽고 물음에 답을 유추하시오. 
#2: 질의응답 태스크 수행: 
#3: {맥락} \n {질문}에 대해 답이 뭔지 아니? 

'''
import gc

for i in range(5774):
  
  gc.collect()
  torch.cuda.empty_cache()
  #index = 2007 + i*20
  index = i
  #prompt_with_query = prompt + '질의응답 태스크 수행:\n\n' + f"맥락: {eval_data[index]['context']} \n 질문: {eval_data[index]['question']} \n 대답:"
  prompt_with_query = prompt +  f"맥락: {eval_data[index]['context']} \n 질문: {eval_data[index]['question']} {promptz} \n 대답:"


  with torch.no_grad():
    tokens = tokenizer.encode(prompt_with_query, return_tensors='pt').to(device='cuda', non_blocking=True)
    gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=len(tokens[0])+5)
    generated = tokenizer.batch_decode(gen_tokens)[0]

  #print(generated[len(prompt):])
  answer_set.append(generated[generated.rfind("대답:")+4:])
  if i % 10 == 0:
    print(i)
  #print(f" Answer: {eval_data[index]['answers']['text']} \n\n")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [ ]:
answer_set_2


[' 바르셀로나 올림픽 [끝]\n\n질문: 소토',
 ' 1, 2, 4, 5, 6 [',
 ' 소토마요르 [끝]\n\n질문:',
 ' 3번째 시도에서 2.36m\n',
 ' 1996년\n 질문: 소토마요르가 올림픽',
 ' 찰스 오스틴 [끝]\n\n질문: 소토',
 ' 2.39m.\n질문: 찰스 오스틴',
 ' 소토마요르는 실외 시즌 동안에 부상을',
 ' 애틀랜타 올림픽 [끝]\n\n질문: 소토',
 ' 7 위. \n질문: 소토마요르',
 ' 찰스 오스틴 [끝]\n\n질문: 소토',
 ' 2000년\n 질문: 소토마요르가 출전',
 ' 출전 정지를 당하고, 예선 탈락하고,',
 ' 2000년 올림픽\n질문: 2000년 올림픽에서',
 ' 소토마요르\n\n질문: 소토마요르',
 ' 은메달\n질문: 소토마요르가 올림픽 높이',
 ' 2.39m(7 피트 7.14',
 ' 2.32m [끝]\n\n질문',
 ' 2015년 11월\n 질문: 조현아 전 부사',
 ' 63억 [끝]\n\n질문: 조현아',
 ' 49억300만 원\n 질문: 2014년',
 ' 조현아 [끝]\n\n질문: 조현아 전',
 ' 싸이버스카이 [끝]\n\n질문:',
 ' 49억300만 원\n 질문: 싸이버스',
 ' 싸이버스카이 [끝]\n\n질문:',
 ' 대한항공\n질문: 싸이버스카이가 대한항공의',
 ' 1964년 5월 5일\n 질문: 김이',
 ' 서울대학교 [끝]\n\n질문: 김이',
 ' 1975년\n 질문: 김이수가 군검찰관',
 ' 1953년\n 질문: 김이수는 언제 죽',
 ' 전라북도 고창군 \n 질문: 김이수가 다닌 학교',
 ' 서울대학교\n\n질문: 김이수는 사법',
 ' 1953년 3월 24일\n 질문: 김',
 ' 광주서중학교 [끝]\n\n질문:',
 ' 1980년\n질문: 김이수가 군법무관으로',
 ' 대전지방법원\n 질문: 김이수가 2009년',
 ' 김재원\n질문: 김이수는 누구의',
 ' 2017년 5월 19일 [끝]\n',
 ' 1982년 [끝]\n\n질문: 김이'

In [ ]:
import pickle

filePath = './answer_set_prompt5_new_prompt.txt'
#testList = ['123', '456', 'abc', 'def']
with open(filePath, 'wb') as lf:
    pickle.dump(answer_set, lf)



with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)
    print(readList)
    # ['123', '456', 'abc', 'def']


readList

['1989년 2월 15', '임수경이었다.', '1989년 2월 15', '서울청량리경찰서', '서울지방경찰청 [끝', '임종석 [끝]\n', '서울올림픽을 방해하기', '허영 [끝]\n', '10차 개헌안 [끝', '89조[EOS]', '조국 [끝]\n', '더글러스 맥아더 [끝]', '리처드 닉슨 밑에서\n', '로널드 레이건과 제럴드', '알렉산더 헤이그 2세(', '더글러스 맥아더 [끝]', '1924년 12월 2', '국무장관\n\n질문', '레이거노믹스 [', '퍼트리샤 앤토이', '1943년 [끝]', '3명\n맥락:', '노터데임 대학교', '퍼트리샤 앤토이', '노터데임 대학교', '제1 기병 사단\n', '퍼트리샤 앤토이', '1978년 [끝]', '닉슨 [끝]\n', '5년 [끝]', '1950년\n\n질문', '나토[EOS]', '1973년\n\n질문', '스틸 [끝]\n', '외교정책의 아마추어\n', '1982년 [끝]', '국제적 상담 회사 [', '1945년\n\n질문', '외교 정책의 아마추어 [', '1981년\n질문:', '노아 [끝]\n', '마태복음 [끝]', '잣나무\n\n질문', '하나님  ·  하나', '3층\n\n질문', '역청\n\n맥락', '약 45cm\n질문', '1층 [끝]', '잣나무\n\n질문', '기독교 [끝]\n', '노아의 대홍수가', '조렌느 [끝', '근본주의 개신교 [끝]', '노아의 방주 [끝', '근본주의계열의 개신교 [', '과학 [끝]\n', '홍수지질학 [', '남침례교 [끝', '조렌느 [끝', '유사지질학 [끝', '성경에 기록된 대로', '성경의 노아의 방주', '이슬람교 경전\n\n질문', '노아 방주 [끝]', '제칠일안식', 'Fernand Navarra', '중국 [끝]\n', ' 노아의 방주가', '노아의 방주 [끝', '노아의 방주 [끝', '중국 [끝]\n', '4종\n\n질문', '2억 년 [끝', '염분이나 수온으로 인해', '북극곰이라는 특성이', '노아의 홍수

['1989년 2월 15',
 '임수경이었다.',
 '1989년 2월 15',
 '서울청량리경찰서',
 '서울지방경찰청 [끝',
 '임종석 [끝]\n',
 '서울올림픽을 방해하기',
 '허영 [끝]\n',
 '10차 개헌안 [끝',
 '89조[EOS]',
 '조국 [끝]\n',
 '더글러스 맥아더 [끝]',
 '리처드 닉슨 밑에서\n',
 '로널드 레이건과 제럴드',
 '알렉산더 헤이그 2세(',
 '더글러스 맥아더 [끝]',
 '1924년 12월 2',
 '국무장관\n\n질문',
 '레이거노믹스 [',
 '퍼트리샤 앤토이',
 '1943년 [끝]',
 '3명\n맥락:',
 '노터데임 대학교',
 '퍼트리샤 앤토이',
 '노터데임 대학교',
 '제1 기병 사단\n',
 '퍼트리샤 앤토이',
 '1978년 [끝]',
 '닉슨 [끝]\n',
 '5년 [끝]',
 '1950년\n\n질문',
 '나토[EOS]',
 '1973년\n\n질문',
 '스틸 [끝]\n',
 '외교정책의 아마추어\n',
 '1982년 [끝]',
 '국제적 상담 회사 [',
 '1945년\n\n질문',
 '외교 정책의 아마추어 [',
 '1981년\n질문:',
 '노아 [끝]\n',
 '마태복음 [끝]',
 '잣나무\n\n질문',
 '하나님  ·  하나',
 '3층\n\n질문',
 '역청\n\n맥락',
 '약 45cm\n질문',
 '1층 [끝]',
 '잣나무\n\n질문',
 '기독교 [끝]\n',
 '노아의 대홍수가',
 '조렌느 [끝',
 '근본주의 개신교 [끝]',
 '노아의 방주 [끝',
 '근본주의계열의 개신교 [',
 '과학 [끝]\n',
 '홍수지질학 [',
 '남침례교 [끝',
 '조렌느 [끝',
 '유사지질학 [끝',
 '성경에 기록된 대로',
 '성경의 노아의 방주',
 '이슬람교 경전\n\n질문',
 '노아 방주 [끝]',
 '제칠일안식',
 'Fernand Navarra',
 '중국 [끝]\n',
 ' 노아의 방주가',
 '노아의 방주 [끝',
 '노아의 방주 [끝',
 '중

## RUN If your RAM is down.. 

In [ ]:

import gc
gc.collect()
torch.cuda.empty_cache()



# Evaluation

## Inference Score with F1-as metric

- answer #1: prompt[0], with golden label, with k=4 
- answer #2: prompt[0], with random label, with k=4 
- answer #3: zeroshot ,no prompt, with k=4 
- answer #4: prompt[1], with golden label ,with k=4 
- answer #5: prompt[2], with golden label, with k=4 




Result #1

올바른 데이터가 중요한가? 
포맷만 맞으면 된다? 

비교: answer #1, #2, #3




Result #2

prompt에 따라 어떻게 변하는가?



비교: result #1, #4, #5 






In [15]:
import pickle


filePath1 = './answer_set_prompt1.txt'

with open(filePath1, 'rb') as lf:
    readList1 = pickle.load(lf)
    print(readList1)
    # ['123', '456', 'abc', 'def']




['1989년 2월 15일        ', '임수경\n 질문: 임종석이 발표한 성명', '1989년\n 질문: 서울지방검찰청 공안부', '서울청량리경찰서\n 질문: 임종석을', '서울지방경찰청 공안분실 [끝]\n', '임종석 [끝]\n\n질문: 임종석', '전국대학생대표자협의회(전대협) 의장으로서', '허영 [끝]\n\n질문: 대표적인', '10차 개헌안 발표 [끝]\n\n질문', '89조 [국무회의의 심의사항]', '허영 [끝]\n\n질문: 정부의', '육군 부참모 총장 [끝]\n\n질문', '국무장관\n 질문: 알렉산더 헤이그는 어떤 직책', '로널드 레이건 [끝]\n\n질문: 알렉산더', '알렉산더 헤이그 2세\n 질문: 알렉산더 헤이그 2', '미 2사단 부참모 총장\n 질문: 에드윈', '1904년 12월 20일 ~ 1980년 1', '유엔대사 [끝]\n\n질문: 알렉산더', "'경고:현실주의, 레이건과 외교 정책'", '퍼트리샤 앤토이넷 폭스 [끝]', '1944년 [1]\n  미국 육군 사관', '3명\n 질문: 알렉산더 헤이그는 일본에서', '노터데임 대학교 [끝]\n\n', '퍼트리샤 앤토이넷 [끝]\n', '노터데임 대학교[끝]\n\n', '제1 기병 사단\n 질문: 그가 훈련', '퍼트리샤 앤토이넷 폭스 [끝]', '1973년\n 질문: 헤이그는 미국 유럽 연합군', '닉슨 [끝]\n\n질문: 알렉산더 헤이그', '3년간 [끝]\n\n질문:', '1975년 [끝]\n질문: 닉슨과', '유럽 연합 [끝]\n\n질문: 헤이그', '1969년\n 질문: 헤이그는 언제 군대에', '미국 유럽 연합군 최고사령부의 최고 사령관 [끝', '외교 정책의 아마추어\n 질문: 알렉산더 헤이그는', '1984년 [끝]\n\n질문: 헤이그', '세계적으로 국제적 상담 회사 [끝]', '1988년            1988년  ', '외교 정책의 아마추어 [끝]\n\n질문', '1988년\n 질문: 헤이그가 대통령 선거에', '노아\n 질문: 노아가 홍수를 피

In [16]:
filePath2 = './answer_set_prompt2.txt'


with open(filePath2, 'rb') as lf:
    readList2 = pickle.load(lf)
    print(readList2)
    # ['123', '456', 'abc', 'def']

['1989년 2월 15일\n 질문: 임종', '임수경이었다.\n질문: 임수경이', '1989년\n 질문: 임종석을 처음으로 발탁', '서울지방경찰청 [끝]\n\n질문:', '서울지방경찰청 공안분실\n\n질문:', '임종석 \n 질문: 1989년 2월', '평화 시위\n 질문: 1989년 3월 12', '허 석좌교수\n\n질문: 개헌안 논의가', '청와대 비서실이 아닌 국무회의 중심으로 이뤄졌어야 했', '제89조(국무회의) 1 국무회의는', '허영 교수\n\n질문: 허영 교수가 지적', '부참모 총장 [끝]\n\n질문:', '국방장관\n 질문: 알렉산더 헤이그와 로널드 레이건', '로널드 레이건 [끝]\n\n질문: 알렉산더', '알렉산더 헤이그 2세. [끝]\n\n', '육군 부참모 총장\n 질문: 알렉산더 헤이그 2', '1924년 12월 2일\n 질문: 알렉산더', '국무장관\n 질문: 알렉산더 헤이그는 어느 나라', '경고:현실주의, 레이건 외교 정책\n\n질문', '퍼트리샤 앤토이넷 폭스\n\n질문', '1944년\n질문: 알렉산더 헤이그가 임관된', '4명 [끝]\n\n질문: 알렉산더', '노터데임 대학교\n질문: 그가', '퍼트리샤 앤토이넷 폭스\n\n질문', '노터데임 대학교\n 질문: 알렉산더 헤이그', '제1 기병 사단\n 질문: 그는 소위', '퍼트리샤 앤토이넷 폭스\n\n질문', '1975년\n 질문: 헤이그의 전역은 언제', '닉슨 대통령\n 질문: 알렉산더 헤이그는 닉슨 대통령', '6개월 [끝]\n\n질문: 헤이그', '1973년\n 질문: 8월 닉슨 사임과', '유럽 연합 최고사령부 [끝]\n\n질문', '1977년\n질문: 헤이그는 몇 군대에서', '기술 주식 [끝]\n\n질문: 어느', '외교 정책의 아마추어 [끝]\n\n질문', '1982년\n 질문: 헤이그는 자신이 레이', '세계일보 [끝]\n\n질문: 헤이그', '1981년, 8년  질문: 1981년', '아마추어 외교 정책 [끝]\n\n질문:', '1981년\n 질문: 

In [17]:
filePath = './answer_set_prompt3_nodemo.txt'


with open(filePath, 'rb') as lf:
    readList3 = pickle.load(lf)
    print(readList3)
    # ['123', '456', 'abc', 'def']

['1989년 2월 15', '임수경이었다.', '1988년\n 질문:', '1989년 12월 18', '1990년 1월 30', '임종석이다.\n', '1989년 2월 15', '허영\n=== *헌법', '1국민투표법 개정안', '"국무회의의 심의', '허영 교수\n\n\n', '미국 국방부 부(副', '헤이그는 레이건 정부 출범', '로널드 레이건.\n 질문', '레이건 대통령은 1980', '2번째\n질문', '1924년 12월 2', '    \n 헤이그는', '"경고:현실주의,', '퍼트리샤 앤토이', '1944년이다.\n', '3명이다.\n', '미국 노터데임', '퍼트리샤 앤토이', '노터데임 대학교', '웨스트포인트가 첫번', '퍼트리샤 앤토이', '1975년이다. 그', '알렉산더 헤이그는 미국에서', '헤이그는 1964년부터', '1973년입니다.\n', '그는 1981년 미국', '닉슨이 대통령으로 선출', '최고 운영 책임자였다', '그들은 국내적', '그는 1978년 12', 'Worldwide Associates Inc', '1982년입니다. 질문', '외교 정책의 아마추어.', '1982년  질문:', '노아의 직업은 농업', ' 마태복음 24장', '노아의 방주가 홍', '여호와께서 "물이', '배가 건조되어', '역청(아스팔트', '1규빗=약', '위에서 언급한 바', '고페르나무: 역', '대부분의 개신교이다.', '현대 과학의 발달로', '첫째, 근본주의적 시각', '가톨릭의 개신교와 근본주의', '현대 대부분의 기독교에서', '개신교, 가톨릭, 그리고', '세계적인 홍수가 실재', '홍수지질학\n', '극보수주의계열의', '고대사회에서 성경은', '유사지질학으로 남', '노아시대에 노아는', '유사지질학인 홍', '유사지질학자들은', '증거,      -', '한국창조과학회 등이', 'Fernand Navarra', 'BTV(중국).', '노아의 방주를 본', '중국에서 발견되었', '네, 그렇습니다.', '이스라엘\n *창

In [18]:
filePath = './answer_set_prompt4_new_prompt.txt'


with open(filePath, 'rb') as lf:
    readList4 = pickle.load(lf)
    print(readList4)
    # ['123', '456', 'abc', 'def']

['1989년 2월 15', '임수경전무처장 ', '1989년 [끝]', '서울 장안동 [끝', '장안동 [끝]', '임종석이다.\n', '1989년 2월 15', '허영 [끝]\n', '10차 개헌안 발표\n', '제89조\n\n', '허영 [끝]\n', '미국 육군부참모 총장', '국무 장관.\n질문', '로널드 레이건 [끝]', '알렉산더 헤이그\n\n질문', '더글러스 맥아더 유엔군 사령부 참모', '1890년 5월 23', '국무장관 [끝]', '경고  현실주의  레이', '퍼트리샤 앤토이', '1945년\n 질문:', '3명\n 질문:', '노터데임 대학교', '퍼트리샤 앤토이', '노터데임 대학교', '포트녹스에서 기갑 훈련', '퍼트리샤 앤토이', '1974년 [끝]', '닉슨 [끝]\n', '5년 [끝]', '1979년 [끝]', '유럽 연합군 최고사령부 [', '1973년 [끝]', '미국 기술 주식 회사 [', '외교 정책의 아마추어 [', '1982년 [끝]', 'WBAI [끝', '1987년과 1988년', '아마추어 외교 정책가들', '1988년 [끝]', '노아 [끝]  ', '〈창세기〉 6', '역청이라는 아스팔트와', '사람 [끝]\n', '3층\n  질문', '역청 [끝]', '1규빗 = 약', '3층으로 되어', '잣나무 [끝]', '기독교 [끝]\n', '현대 과학의 발달로', '노아의 방주 이야기는', '보수적 개신교 계열로', '상징적 의미로 받아들인다', '개신교(가톨릭 포함)', '과학과 역사학\n\n', '홍수지질학 [', '남침례교 [끝', '성경에 나오는 노아', '유사지질학[끝', '일반적으로 근본주의적', '노아의 홍수가 실제로', '길가메쉬 서사시 [', '전혀 존재하지 않', '한국창조과학회 [', 'Fernand Navarra', 'BTV [끝]', '노아 방주는 목재로', '유사지질학으로 평가', '5개 연구소에서 연대', '중국\n\n질문:', '(생물다양성이란', '모든 진화적 종분', '수생생물류

In [19]:
filePath = './answer_set_prompt5_new_prompt.txt'

with open(filePath, 'rb') as lf2:
    readList5 = pickle.load(lf2)
    print(readList5)
    # ['123', '456', 'abc', 'def']

['1989년 2월 15', '임수경이었다.', '1989년 2월 15', '서울청량리경찰서', '서울지방경찰청 [끝', '임종석 [끝]\n', '서울올림픽을 방해하기', '허영 [끝]\n', '10차 개헌안 [끝', '89조[EOS]', '조국 [끝]\n', '더글러스 맥아더 [끝]', '리처드 닉슨 밑에서\n', '로널드 레이건과 제럴드', '알렉산더 헤이그 2세(', '더글러스 맥아더 [끝]', '1924년 12월 2', '국무장관\n\n질문', '레이거노믹스 [', '퍼트리샤 앤토이', '1943년 [끝]', '3명\n맥락:', '노터데임 대학교', '퍼트리샤 앤토이', '노터데임 대학교', '제1 기병 사단\n', '퍼트리샤 앤토이', '1978년 [끝]', '닉슨 [끝]\n', '5년 [끝]', '1950년\n\n질문', '나토[EOS]', '1973년\n\n질문', '스틸 [끝]\n', '외교정책의 아마추어\n', '1982년 [끝]', '국제적 상담 회사 [', '1945년\n\n질문', '외교 정책의 아마추어 [', '1981년\n질문:', '노아 [끝]\n', '마태복음 [끝]', '잣나무\n\n질문', '하나님  ·  하나', '3층\n\n질문', '역청\n\n맥락', '약 45cm\n질문', '1층 [끝]', '잣나무\n\n질문', '기독교 [끝]\n', '노아의 대홍수가', '조렌느 [끝', '근본주의 개신교 [끝]', '노아의 방주 [끝', '근본주의계열의 개신교 [', '과학 [끝]\n', '홍수지질학 [', '남침례교 [끝', '조렌느 [끝', '유사지질학 [끝', '성경에 기록된 대로', '성경의 노아의 방주', '이슬람교 경전\n\n질문', '노아 방주 [끝]', '제칠일안식', 'Fernand Navarra', '중국 [끝]\n', ' 노아의 방주가', '노아의 방주 [끝', '노아의 방주 [끝', '중국 [끝]\n', '4종\n\n질문', '2억 년 [끝', '염분이나 수온으로 인해', '북극곰이라는 특성이', '노아의 홍수

In [21]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.1 MB/s 
     |████████████████████████████████| 212 kB 43.4 MB/s 
     |████████████████████████████████| 140 kB 79.7 MB/s 
     |████████████████████████████████| 1.1 MB 70.2 MB/s 
     |████████████████████████████████| 101 kB 15.9 MB/s 
     |████████████████████████████████| 596 kB 71.0 MB/s 
     |████████████████████████████████| 127 kB 80.1 MB/s 
     |████████████████████████████████| 271 kB 74.8 MB/s 
     |████████████████████████████████| 144 kB 76.4 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

In [50]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5

'''
from datasets import load_dataset

#dataset = load_dataset("klue", "mrc")


dataset = load_dataset("squad_kor_v1")

answer = dataset['validation']['answers']



for i in range(len(answer)):
  try:
    answer[i] = answer[i]['text'][0]
  except:
    print(i)


#answer


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
#readList1

def preprocessing(text):
  if text.find('[끝]:') != -1:
    text = text[:text.find('[끝]:')]
  if text.find('[끝:') != -1:
    text = text[:text.find('[끝:')]
  if text.find('[끝]') != -1:
    text = text[:text.find('[끝]')]
  if text.find('[끝') != -1:
    text = text[:text.find('[끝')]
  if text.find('질문:') != -1:
    text = text[:text.find('질문:')]
  
  return text





for i in range(len(readList1)):
  readList1[i] = preprocessing(readList1[i])

for i in range(len(readList2)):
  readList2[i] = preprocessing(readList2[i])

for i in range(len(readList3)):
  readList3[i] = preprocessing(readList3[i])

for i in range(len(readList4)):
  readList4[i] = preprocessing(readList4[i])

for i in range(len(readList5)):
  readList5[i] = preprocessing(readList5[i])




print(readList5)


['1989년 2월 15', '임수경이었다.', '1989년 2월 15', '서울청량리경찰서', '서울지방경찰청 ', '임종석 ', '서울올림픽을 방해하기', '허영 ', '10차 개헌안 ', '89조[EOS]', '조국 ', '더글러스 맥아더 ', '리처드 닉슨 밑에서\n', '로널드 레이건과 제럴드', '알렉산더 헤이그 2세(', '더글러스 맥아더 ', '1924년 12월 2', '국무장관\n\n질문', '레이거노믹스 [', '퍼트리샤 앤토이', '1943년 ', '3명\n맥락:', '노터데임 대학교', '퍼트리샤 앤토이', '노터데임 대학교', '제1 기병 사단\n', '퍼트리샤 앤토이', '1978년 ', '닉슨 ', '5년 ', '1950년\n\n질문', '나토[EOS]', '1973년\n\n질문', '스틸 ', '외교정책의 아마추어\n', '1982년 ', '국제적 상담 회사 [', '1945년\n\n질문', '외교 정책의 아마추어 [', '1981년\n', '노아 ', '마태복음 ', '잣나무\n\n질문', '하나님  ·  하나', '3층\n\n질문', '역청\n\n맥락', '약 45cm\n질문', '1층 ', '잣나무\n\n질문', '기독교 ', '노아의 대홍수가', '조렌느 ', '근본주의 개신교 ', '노아의 방주 ', '근본주의계열의 개신교 [', '과학 ', '홍수지질학 [', '남침례교 ', '조렌느 ', '유사지질학 ', '성경에 기록된 대로', '성경의 노아의 방주', '이슬람교 경전\n\n질문', '노아 방주 ', '제칠일안식', 'Fernand Navarra', '중국 ', ' 노아의 방주가', '노아의 방주 ', '노아의 방주 ', '중국 ', '4종\n\n질문', '2억 년 ', '염분이나 수온으로 인해', '북극곰이라는 특성이', '노아의 홍수가 실재', '삼투압 때문이라 볼 수', '8.600년\n', '목재 선박의 배수량의', '창조과학회 ', '100m\n\n질문', '창조과학회\n\n', '지질학 석사학위, 생물학',

In [51]:
len(answer)

5774

In [52]:
len(readList1)

5774

In [53]:
len(readList2)

5774

In [55]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5


answer 
'''


f1_1 = 0
f1_2 = 0
f1_3 = 0
f1_4 = 0
f1_5 = 0


for i in range(len(answer)):
  f1_1 += f1_score(answer[i],readList1[i])
  f1_2 += f1_score(answer[i],readList2[i])
  f1_3 += f1_score(answer[i],readList3[i])
  f1_4 += f1_score(answer[i],readList4[i])
  f1_5 += f1_score(answer[i],readList5[i])



print(f1_1)
print(f1_2)
print(f1_3)
print(f1_4)
print(f1_5)





print(f1_1*100/len(answer))
print(f1_2*100/len(answer))
print(f1_3*100/len(answer))
print(f1_4*100/len(answer))
print(f1_5*100/len(answer))




3631.1922681880715
3237.1069079424506
2384.6635907522973
3428.9839656938466
3108.515710737004
62.88867800810654
56.0635072383521
41.300027550264936
59.38662912528311
53.836434200502325


In [65]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5


answer 
'''


em_1 = 0
em_2 = 0
em_3 = 0
em_4 = 0
em_5 = 0


for i in range(len(answer)):
  em_1 += em_score(answer[i],readList1[i])
  em_2 += em_score(answer[i],readList2[i])
  em_3 += em_score(answer[i],readList3[i])
  em_4 += em_score(answer[i],readList4[i])
  em_5 += em_score(answer[i],readList5[i])



print(em_1)
print(em_2)
print(em_3)
print(em_4)
print(em_5)





print(em_1*100/len(answer))
print(em_2*100/len(answer))
print(em_3*100/len(answer))
print(em_4*100/len(answer))
print(em_5*100/len(answer))




2411
1881
667
2023
1783
41.756148250779354
32.57706962244544
11.551783858676828
35.03636993418774
30.879806027017665


In [ ]:
'''
#1 : readList1
#2 : readList2
#3 : readList3
#4 : readList4
#5 : readList5


answer 


Archive: 자체적인 메트릭으로 score를 내린 결과 -> 논문에는 f1-score/exact-match를 메트릭으로 하는 것이 훨씬 적절함 


f1_1 = len(answer)  
f1_2 = len(answer)
f1_3 = len(answer)
f1_4 = len(answer)
f1_5 = len(answer) 


j = '안중EEWEz'
j.lower() #안중eewez




#Lower, delete all spaces,
for i in range(len(answer)):
  readList1[i] =readList1[i].lower().strip()
  readList2[i] =readList2[i].lower().strip()
  readList3[i] =readList3[i].lower().strip()
  readList4[i] =readList4[i].lower().strip()
  readList5[i] =readList5[i].lower().strip()



for i in range(len(answer)):
  if readList1[i].find(answer[i]) == -1 and answer[i].find(readList1[i]) == -1: f1_1 = f1_1 -1 
  if readList2[i].find(answer[i]) == -1 and answer[i].find(readList2[i]) == -1: f1_2 = f1_2 -1 
  if readList3[i].find(answer[i]) == -1 and answer[i].find(readList3[i]) == -1: f1_3 = f1_3 -1 
  if readList4[i].find(answer[i]) == -1 and answer[i].find(readList4[i]) == -1: f1_4 = f1_4 -1 
  if readList5[i].find(answer[i]) == -1 and answer[i].find(readList5[i]) == -1: f1_5 = f1_5 -1 



print(f1_1)
print(f1_2)
print(f1_3)
print(f1_4)
print(f1_5)



print(f1_1*100/len(answer))
print(f1_2*100/len(answer))
print(f1_3*100/len(answer))
print(f1_4*100/len(answer))
print(f1_5*100/len(answer))



'''




2943
2543
2050
2666
2293
50.969864911673014
44.0422583997229
35.50398337374437
46.172497402147556
39.71250432975407


In [63]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("\n", " ", text)
        text = re.sub("\[", " ", text)
        text = re.sub("\]", " ", text)      
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
   
    #F1 by character
    prediction_Char = []
    for tok in prediction_tokens:
        now = [a for a in tok]
        prediction_Char.extend(now)
        
    ground_truth_Char = []
    for tok in ground_truth_tokens:
        now = [a for a in tok]
        ground_truth_Char.extend(now)   
        
    common = Counter(prediction_Char) & Counter(ground_truth_Char)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    
    precision = 1.0 * num_same / len(prediction_Char)
    recall = 1.0 * num_same / len(ground_truth_Char)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1




def em_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


In [11]:
x = f1_score('보수적 근본주의 계열 개신교','보수적 근본주의계열의 개신교')

In [12]:
x

0.9600000000000001

In [25]:
x = f1_score('\'1869년 메이지 2년 3월 25일\'','1879년 메이지 2년 3월 25일')

In [26]:
x

0.9333333333333333